# Unifying two datasets into one

- Football-Player-Detection (v8-resized1280_tile2x2_aug3x.yolov5pytorch)
    - Classes: football, player (one image marked null)
    - Resized to 1280x1280 (Stretch)
    - Normalised bb
- SOD_Dataset
    - Classes: ball, player
    - Annotations: For each image, there is a text file containing the class ID, Xmin, Ymin, Xmax, and Ymax, respectively.
    - yolov5_annotations: Box coordinates are in normalized xywh format (from 0 - 1) for yolov5_annotations. x_center and width are divided by image width, and y_center and height by image height.

 
 # Final dataset
 - Classes: football, player
 - Resized to 1280x1280 (Stretch)
 - Annotations: For each image, there is a text file containing the class ID, Xmin, Ymin, Xmax, and Ymax, respectively. (bottom left, top right corners ?¿)

***
## Important Note
The two oringinal datasets must be downloaded into a "Datasets" folder in the parent folder of the repository:

Download from:
- https://github.com/FootballAnalysis/footballanalysis/tree/main/Dataset/Object%20Detection%20Dataset
- https://universe.roboflow.com/augmented-startups/football-player-detection-kucab

And name, respectively:
- SOD_Dataset
- Football-Player-Detection
***

In [1]:
import os
import torch
from PIL import Image
from torchvision import transforms
import numpy as np


c:\Users\Usuario\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\detection\anchor_utils.py:63: UserWarning: Failed to initialize NumPy: module compiled against API version 0x10 but this version of numpy is 0xf . Check the section C-API incompatibility at the Troubleshooting ImportError section at https://numpy.org/devdocs/user/troubleshooting-importerror.html#c-api-incompatibility for indications on how to solve this problem . (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:84.)
  device: torch.device = torch.device("cpu"),


# Reshaping SOD_Dataset and switching labels


In [6]:
from PIL import Image
import os

# Set the path to your image directory
image_dir = "Datasets/SOD_Dataset/images"
image_output_dir = "Datasets/SOD_Dataset/resized_images"
os.makedirs(image_output_dir, exist_ok = True)

# Set the path to your label directory
label_dir = "Datasets/SOD_Dataset/annotations"
label_output_dir = "Datasets/SOD_Dataset/resized_annotations"
os.makedirs(label_output_dir, exist_ok = True)

new_size = (1280, 1280)

# Loop through each image file in the directory
for filename in os.listdir(image_dir):
    # Open the image file using PIL
    img = Image.open(os.path.join(image_dir, filename))
    
    # Get the size of the image
    # width, height = img.size
    # Print the image size
    # print("Image size for {}: {} x {}".format(filename, width, height))

    # Resize image
    new_image = img.resize(new_size)
    new_image.save(os.path.join(image_output_dir, filename))

    # Open the corresponding label file
    label_filename = os.path.splitext(filename)[0] + ".txt"
    label_path = os.path.join(label_dir, label_filename)
    with open(label_path, "r") as f:
        label_data = f.readlines()
     # Resize the bounding boxes in the label file
    new_label_data = []
    for line in label_data:
        parts = line.strip().split(" ")
        class_id = parts[0]
        bounding_box = parts[1:]

        x_min = float(bounding_box[0])
        y_min = float(bounding_box[1])
        x_max = float(bounding_box[2])
        y_max = float(bounding_box[3])
        
        # Resize the bounding box coordinates
        x_min_resized = int(x_min * (new_size[0] / img.width))
        y_min_resized = int(y_min * (new_size[1] / img.height))
        x_max_resized = int(x_max * (new_size[0] / img.width))
        y_max_resized = int(y_max * (new_size[1] / img.height))
        
        # Change class id to match other dataset:
        new_class_id = class_id # in case there is any null 
        if class_id == '0':
            new_class_id = '1'
        elif class_id == '1': 
            new_class_id = '0'
        
        new_parts = [new_class_id, str(x_min_resized), str(y_min_resized), str(x_max_resized), str(y_max_resized)]
        new_label_data.append(" ".join(new_parts))
    
        output_label_filename = os.path.splitext(filename)[0] + ".txt"
        output_label_path = os.path.join(label_output_dir, output_label_filename)
        with open(output_label_path, "w") as f:
            f.write("\n".join(new_label_data))



        

# Modifying Bounding boxes in  Football-Player-Detection to correct format and joining datasets

In [9]:
from PIL import Image
import os
import shutil

# Set the path to your image directory
image_directories = ["Datasets/Football-Player-Detection/train/images", "Datasets/Football-Player-Detection/test/images", "Datasets/Football-Player-Detection/valid/images"]
image_output_dir = "Datasets/Football-Player-Detection/resized_images"
os.makedirs(image_output_dir, exist_ok = True)
# Set the path to your label directory
label_dirrectories = ["Datasets/Football-Player-Detection/train/labels", "Datasets/Football-Player-Detection/test/labels", "Datasets/Football-Player-Detection/valid/labels"]
label_output_dir = "Datasets/Football-Player-Detection/resized_annotations"
os.makedirs(label_output_dir, exist_ok = True)


for image_dir, label_dir in zip(image_directories, label_dirrectories) :
    # Loop through each image file in the directory
    for filename in os.listdir(image_dir):
        #  save in new folder MAYBE LATER ALSO RESIZE; BUT THIS IS FASTER NOW
        #img = Image.open(os.path.join(image_dir, filename))
        #img.save(os.path.join(image_output_dir, filename))
        original_im_dir = os.path.join(image_dir, filename)
        target_im_dir = os.path.join(image_output_dir, filename)

        shutil.move(original_im_dir, target_im_dir)

        # Open the corresponding label file
        label_filename = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(label_dir, label_filename)
        with open(label_path, "r") as f:
            label_data = f.readlines()
        # Resize the bounding boxes in the label file
            # Resize the bounding boxes in the label file
        new_label_data = []
        for line in label_data:
            parts = line.strip().split(" ")
            x_center = float(parts[1])
            y_center = float(parts[2])
            box_width = float(parts[3])
            box_height = float(parts[4])
            
            # Convert normalized coordinates to pixel coordinates
            x_min = int((x_center - (box_width / 2)) * img.width)
            y_min = int((y_center - (box_height / 2)) * img.height)
            x_max = int((x_center + (box_width / 2)) * img.width)
            y_max = int((y_center + (box_height / 2)) * img.height)
            
            new_parts = [parts[0], str(x_min), str(y_min), str(x_max), str(y_max)]
            new_label_data.append(" ".join(new_parts))
        
            output_label_filename = os.path.splitext(filename)[0] + ".txt"
            output_label_path = os.path.join(label_output_dir, output_label_filename)
            with open(output_label_path, "w") as f:
                f.write("\n".join(new_label_data))

        

# Using Dan's Framework 

In [10]:
from torchvision.io import read_image, ImageReadMode
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from PIL import Image
import os
import torch



# this function is for the DataLoader, it makes sure the tensors within a batch are the same dimension (don't ask how, I don't know)
def collate_fn(batch):
    return tuple(zip(*batch))


# this is a class that loads the data, according to how pytorch wants it
class playersDataset(Dataset): # dataset is the path
    def __init__(self, folder_path, img_size=416):
        self.root = folder_path
        self.images_folder = os.path.join(folder_path, "resized_images")
        self.labels_folder = os.path.join(folder_path, "resized_annotations")
        
        self.image_files = os.listdir(self.images_folder)
        self.label_files = os.listdir(self.labels_folder)

        self.convert_tensor = transforms.ToTensor()

    def readLabelsFile(self, file_path, index):
        boxes = []
        labels = []
        areas = []
 
        with open(file_path) as f:
            for row in f:
                annotation = [float(x) for x in row.split()]
                #print(annotation)
                labels.append(int(annotation[0]))
                [x0, y0, x1, y1] = annotation[1:5]
                boxes.append([x0, y0, x1, y1])

        # convert everything into a torch.Tensor
        boxes = torch.as_tensor(boxes, dtype=torch.float32)         
        labels = torch.as_tensor(labels, dtype=torch.int64)

        areas = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        iscrowd = torch.zeros((len(labels),), dtype=torch.int64)

        target = {
            "boxes" : boxes,
            "labels" : labels,
            "image_id" : torch.tensor(index),
            "area" : areas,
            "iscrowd" : iscrowd
            }

        return target


    # pytorch needs this, it returns a single (image, output) pair
    def __getitem__(self, index):
        # load and format the image file as a tensor
        
        imgPath = os.path.join(self.images_folder, self.image_files[index])
        img = Image.open(imgPath)

        # TEMPORARY - REMOVE THIS LATER, WE SHOULD DECIDE HOW LARGE THE IMAGES ARE
        img = img.resize((1080, 1920))

        input_img = self.convert_tensor(img)

        # load and format the corresponding labels
        labelPath = os.path.join(self.labels_folder, self.label_files[index])
        target = self.readLabelsFile(labelPath, index)
        
        return input_img, target

    # pytorch also needs the length of the dataset
    def __len__(self):
        return len(self.image_files)


# Unifying datasets

In [11]:
from torch.utils.data import ConcatDataset
dataset1_dir =  "Datasets/SOD_Dataset"
dataset2_dir = "Datasets/Football-Player-Detection"

dataset1 = playersDataset(dataset1_dir)
dataset2 = playersDataset(dataset2_dir)
# Unifying datasets
dataset = ConcatDataset([dataset1, dataset2])



# Save the new DATASET

In [12]:
# Create new directory for concatenated dataset
new_dataset_dir = "Datasets/Concatenated_Dataset"
os.makedirs(new_dataset_dir, exist_ok = True)

# Loop over each image and its corresponding label file
for i, (img, boxes, labels, _) in enumerate(dataset):
    # Save image to new directory
    img_filename = f"img_{i}.jpg"
    img_path = os.path.join(new_dataset_dir, img_filename)
    img.save(img_path)
    
    # Save label file to new directory
    label_filename = f"img_{i}.txt"
    label_path = os.path.join(new_dataset_dir, label_filename)
    
    # Rescale bounding boxes to match new image size
    boxes[:, [0, 2]] *= img.width
    boxes[:, [1, 3]] *= img.height
    
    # Save label file with rescaled bounding boxes
    with open(label_path, "w") as f:
        for box, label in zip(boxes, labels):
            f.write(f"{label} {box[0]:.4f} {box[1]:.4f} {box[2]:.4f} {box[3]:.4f}\n")

RuntimeError: Numpy is not available

# Train/Validation/Test

In [ ]:
from torch.utils.data import DataLoader, SubsetRandomSampler

batch_size = 16

# Define the size of each set
train_size = int(0.7 * len(dataset))
val_size = int(0.2 * len(dataset))
test_size = len(dataset) - train_size - val_size

# Define the random samplers for each set
train_sampler = SubsetRandomSampler(range(train_size))
val_sampler = SubsetRandomSampler(range(train_size, train_size + val_size))
test_sampler = SubsetRandomSampler(range(train_size + val_size, len(dataset)))

# Define the dataloaders for each set
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler)
val_loader = DataLoader(dataset, batch_size=batch_size, sampler=val_sampler)
test_loader = DataLoader(dataset, batch_size=batch_size, sampler=test_sampler)

# Trying to visualize an image with the bounding boxes 
(error NumPy is not available, idk whats going on yet)

In [ ]:
import torch
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

# Define the transform to convert the tensor back to a PIL Image
to_pil = transforms.ToPILImage()

# Get a random batch of images and their corresponding bounding boxes
torch.tensor(np.array(img,dtype=np.float64))/255.0
images, boxes, labels, _ = next(iter(train_loader))

# Convert the image and bounding boxes to PIL format
image = to_pil(images[0])
boxes = boxes[0]

# Plot the image with the bounding boxes
fig, ax = plt.subplots(1)
ax.imshow(image)

# Loop over each bounding box and draw it on the image
for box in boxes:
    # Convert the normalized coordinates to pixel coordinates
    x, y, w, h = box.tolist()
    x1 = x - w/2
    y1 = y - h/2
    x2 = x + w/2
    y2 = y + h/2
    
    # Draw the bounding box on the image
    rect = plt.Rectangle((x1, y1), w, h, fill=False, edgecolor='r', linewidth=2)
    ax.add_patch(rect)

# Show the image
plt.show()


RuntimeError: Numpy is not available

# not used

In [ ]:
class FootballDataset(torch.utils.data.Dataset):
    def __init__(self, img_folder, ann_folder):
        self.img_folder = img_folder
        self.ann_folder = ann_folder
        self.img_filenames = os.listdir(img_folder)

    def __getitem__(self, idx):
        # Read image
        img_filename = self.img_filenames[idx]
        img_path = os.path.join(self.img_folder, img_filename)
        img = Image.open(img_path)

        # Read annotation
        ann_filename = os.path.splitext(img_filename)[0] + '.txt'
        ann_path = os.path.join(self.ann_folder, ann_filename)
        boxes = []
        labels = []
        with open(ann_path, 'r') as f:
            for line in f:
                box = list(map(float, line.strip().split()))
                labels.append(int(box[0]))
                boxes.append(box[1:])

        # Convert to PyTorch tensor
        img = transforms.ToTensor()(img)
        boxes = torch.tensor(boxes)
        labels = torch.tensor(labels)

        return img, boxes, labels

    def __len__(self):
        return len(self.img_filenames)